In [1]:
import sys
sys.path.append('/home/pierre/riken/word2vec')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import classification_tools

/home/pierre/.conda/envs/python3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pierre/.conda/envs/python3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pierre/.conda/envs/python3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [31]:
K = 5
RANDOM_STATE = 42

data_path = '/home/pierre/riken/data/riken_data/complete_from_xlsx.tsv'

In [10]:
df = pd.read_csv(data_path, sep='\t')
df.loc[:, 'seq_len'] = df.sequences.apply(len)
df = df.loc[df.seq_len >= 50, :]

sequences, y = df['sequences'].values, df['is_allergenic'].values

# Feature extraction

In [34]:
#TODO: Test this
def overlapping_tokenizer(seq):
    """
    'ABCDE' ==> ['ABC', 'BCE', 'CDE'] 
    """
    return [seq[idx:idx+K] for idx in range(len(seq)-K+1)]

In [63]:
vectorizer = TfidfVectorizer(tokenizer=overlapping_tokenizer, max_features=8000, lowercase=False) 
X = vectorizer.fit_transform(sequences)

In [32]:
vectorizer.vocabulary_[X[0].todense()[0]

SyntaxError: unexpected EOF while parsing (<ipython-input-32-87ff11a5a6c7>, line 1)

# Prediction using SVM

In [68]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

In [69]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)

In [70]:
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(ytest, ypred >= 0.5))
print('ROC AUC SCORE: ', roc_auc_score(ytest, ypred))

             precision    recall  f1-score   support

      False       0.96      0.98      0.97      3166
       True       0.90      0.78      0.84       575

avg / total       0.95      0.95      0.95      3741

ROC AUC SCORE:  0.8832500205993024


In [71]:
(pd.DataFrame({'coef': clf.coef_.reshape(-1), 'name': [pat for pat in vectorizer.vocabulary_.keys()]})
.assign(absolute_val=lambda x: x.coef.abs())
.sort_values('absolute_val', ascending=False))

,coef,name,absolute_val
3746,1.956175e+00,LAEVA,1.956175e+00
982,1.776552e+00,MAAAV,1.776552e+00
4313,1.738604e+00,ALGVG,1.738604e+00
416,1.649686e+00,EYIQH,1.649686e+00
2240,1.640528e+00,ATSSS,1.640528e+00
1544,1.639498e+00,RAALE,1.639498e+00
7040,1.621189e+00,SAVSA,1.621189e+00
5327,1.513816e+00,QRLKE,1.513816e+00
3440,1.497738e+00,GIVLD,1.497738e+00
1846,1.464736e+00,EKGKD,1.464736e+00
